In [ ]:
import pandas as pd
import altair as alt

In [ ]:
# Enabling Altair 
alt.data_transformers.disable_max_rows()

In [ ]:
# Read dataset from URL
url = "https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/building_inventory.csv"
df = pd.read_csv(url)

In [ ]:
# Handle missing values and ensure correct data types
df['Square Footage'] = pd.to_numeric(df['Square Footage'], errors='coerce')
df['Year Acquired'] = pd.to_numeric(df['Year Acquired'], errors='coerce')
df.dropna(subset=['Square Footage', 'Year Acquired'], inplace=True)

In [ ]:
# Visualization 1: Interactive bar chart with tooltips
avg_sqft = df.groupby(['Agency Name', 'Usage Description'])['Square Footage'].mean().reset_index()

# Selection for filtering
agency_filter = alt.binding_select(options=list(avg_sqft['Agency Name'].unique()), name='Filter by Agency: ')
agency_selection = alt.selection_single(fields=['Agency Name'], bind=agency_filter)

bar_chart = alt.Chart(avg_sqft).mark_bar().encode(
    x=alt.X('Square Footage:Q', title='Average Square Footage'),
    y=alt.Y('Usage Description:N', sort='-x', title='Usage Description'),
    color=alt.Color('Usage Description:N', legend=None),
    tooltip=['Agency Name', 'Usage Description', 'Square Footage']
).add_selection(
    agency_selection
).transform_filter(
    agency_selection
).properties(
    title="Average Square Footage by Usage Description (Filtered by Agency)",
    width=600,
    height=400
)



bar_chart.display()
bar_chart.save('interactive_bar_chart.json')

In [ ]:
# Visualization 2: Interactive scatter plot with selection
scatter_chart = alt.Chart(df).mark_circle(size=60).encode(
    x=alt.X('Year Acquired:Q', title='Year Acquired'),
    y=alt.Y('Square Footage:Q', title='Square Footage'),
    tooltip=['Location Name', 'Square Footage', 'Year Acquired'],  # Tooltip interactivity
    color=alt.Color('Agency Name:N', title='Agency Name')
).properties(
    title="Square Footage vs Year Acquired (Interactive)",
    width=600,
    height=400
)

# Adding interactivity: Multi-selection by agency name and click
legend_selection = alt.selection_multi(fields=['Agency Name'], bind='legend', name="Select Agency")
point_selection = alt.selection_single(on='click', empty='all', name="Highlight Point")

interactive_scatter = scatter_chart.add_selection(
    legend_selection
).add_params(
    point_selection
).transform_filter(
    legend_selection
).encode(
    opacity=alt.condition(point_selection, alt.value(1), alt.value(0.3)) 
)

interactive_scatter.display()
interactive_scatter.save('interactive_scatter_plot.json')